In [1]:
import time
import paho.mqtt.client
import paho.mqtt.publish
import numpy as np
from numpy import random
import datetime
import ssl
import sys
import psycopg2
from sqlalchemy import create_engine
import json

In [2]:
host='lallah.db.elephantsql.com'
user ='yirsxctz'
password='AS3sYxp2fpRwAczZa1OZpIzI4etWHM6n'
dbname='yirsxctz'

In [3]:
def myQuery(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        cur.close()
        myConnection.commit()
        return 1

In [4]:
myConnection = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

In [5]:
def on_connect(client, userdata, flags, rc):    
    print('connected (%s)' % client._client_id)
    client.subscribe(topic='Tienda/#', qos = 0)       

def on_message(client, userdata, message):   
    a = json.loads(message.payload)
    #warnings(a)
    insert_db(a)
    print(a)
    print('------------------------------')  

In [6]:
def insert_db(info):
    
    #clientes conocidos y clientes no conocidos
    if(int(info["afiliado"]) == 1):
        query = """INSERT INTO visitas_conocidas (id_cliente_afiliado, id_tienda, puntos, fecha)
                    values ("""+str(info["cliente"])+""",
                            """+str(info["tienda"])+""",
                            """+str(1)+""",
                            '"""+str(info["fecha"])+"""')"""
        myQuery(query)
    
    else:
        query = """INSERT INTO cliente(cedula, nombre, apellido, direccion, telefono)
                    values ('"""+str(info["cedula_cliente"])+"""',
                            """+str(info["nombre_cliente"])+""",
                            """+str(info["apellido_cliente"])+""",
                            """+str(info["direccion_cliente"])+""",
                            '"""+str(info["telefono_cliente"])+"""')"""
        myQuery(query)
    

    
    #compras
    for x in range(0,len(info["productos"])):
        query = """insert into compra (id_factura, id_producto, cantidad_producto,
                                    id_cambio_precio_producto, id_tienda, id_productoxestante)
                values ('"""+str(3)+"""',
                        '"""+str((info["productos"])[x])+"""',
                        '"""+str((info["cantidades"])[x])+"""',
                        '"""+str((info["cambios_precio_productos"])[x])+"""',
                        '"""+str(info["tienda"])+"""',
                        '"""+str((info["productosxestante"])[x])+"""')"""
        
        myQuery(query)

In [7]:
host = "broker.hivemq.com"

client = paho.mqtt.client.Client()

client.on_connect = on_connect

client.message_callback_add('Tienda/EntradaClientes', on_message)

client.connect(host=host) 

client.loop_start()

connected (b'')
Error en el query: column "manuel" does not exist
LINE 3:                             Manuel,
                                    ^

Error en el query: No hay suficientes productos
CONTEXT:  PL/pgSQL function verificar_cantidad_disponible() line 12 at RAISE

{'afiliado': 0, 'cliente': 1, 'tienda': 2, 'fecha': '2020-11-11 15:49:10.638127', 'nombre_cliente': 'Manuel', 'apellido_cliente': 'Argüelles', 'direccion_cliente': 'Caracas', 'cedula_cliente': 90215731, 'telefono_cliente': 52931425, 'id_factura': 1, 'productos': [1], 'cantidades': [3], 'cambios_precio_productos': [2], 'productosxestante': [1]}
------------------------------
Error en el query: column "cantida_maxima_en_estante" does not exist
LINE 2:   VALUES (NEW.id_productoxestante, cantida_maxima_en_estante...
                                           ^
QUERY:  INSERT INTO producto_disponiblexestante(id_productoxestante, cantidad_disponible_producto, fecha)
		VALUES (NEW.id_productoxestante, cantida_maxima_en_est